[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/abdn-cs3033-ai/practicals/blob/main/week05/tutorial4-csp.ipynb)

# CS3033: Artificial Intelligence

## Tutorial 04: Constraint Satisfaction Problems

#### Prof. Felipe Meneguzzi

In order to run this tutorial, you need to download the auxiliary files from Github into your notebook, which we do with Jupyter's shell commands (if you downloaded the entire repo, the code below is not necessary).

In [3]:
try:
    import google.colab
    print("We are in Google colab, we need to clone the repo")
    !git clone https://github.com/abdn-cs3033-ai/practicals.git
    %cd practicals/week05
    %pip install -r requirements.txt
except:
    print("Not in colab")

We are in Google colab, we need to clone the repo
Cloning into 'practicals'...
remote: Enumerating objects: 324, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 324 (delta 27), reused 36 (delta 19), pack-reused 269 (from 1)
Receiving objects: 100% (324/324), 11.37 MiB | 16.48 MiB/s, done.
Resolving deltas: 100% (156/156), done.
/content/practicals/week05
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.1/29.1 MB 75.0 MB/s eta 0:00:00


Key to this tutorial is to learn how to encode problems in terms of variables and constraints. For this, we will use the [Z3 Theorem Prover](https://github.com/Z3Prover/z3/wiki) from Microsoft Research ([Z3](https://github.com/Z3Prover/z3) is open source and free to use).
If you installed the ```requirements.txt``` file, the following code is not necessary.
```bash
!pip install z3-solver
```

## N-Queens

We return to the [N-Queens Puzzle](https://en.wikipedia.org/wiki/Eight_queens_puzzle).
You will recall that this is the problem of placing $n$ chess queens on a chessboard so that no two queens threaten each other; thus, a solution requires that no two queens share the same row, column, or diagonal. The problem is to put $n$ non-attacking queens on an $n \times n$ chessboard, for which solutions exist for all natural numbers $n$ except for $n = 2$ and $n = 3$. Below we show an example of a solution using iterative improvement, which we implemented in the previous tutorial.

![n-Queens](img/nqueens-example.svg "A sequence of valid moves for sliding-puzzle's navigation problem")

For this ungraded tutorial, we will implement the constraints that describe this problem and let the CSP solver do the hard work for us.

We already provide you with an API for the N-Queens problem, which you can explore in [nqueens.py](../week04/nqueens.py), if you want to refresh your knowledge.

## N-Queens as a CSP

Below, you will fill in the code for the constraints for the N-Queens problem using three types of constraints. Your code will be more readable if you can use Python's [list comprehensions](http://docs.python.org/tutorial/datastructures.html#list-comprehensions). We encode this problem exactly like we did in the local search tutorial. This means we use a python array of integers where each index represents the column of a queen on the board, and the number in each position of the array represents the row we place each queen. Thus, the solution to the example above would be encoded in the array ```[1, 3, 0, 2]```.
The variables you will use are as follows:

- ```n_queens``` represents the number of queens of our problem
- ```Q``` represents the array, in Z3 [Integer](https://z3prover.github.io/api/html/z3.z3.html#-Int) variables
- ```val_c``` represents the constraints on the value of each index in our array, you will need an array of constraints, each of which containing a [disjunction](https://z3prover.github.io/api/html/z3.z3.html#-And) of the numeric range. For example, if you want a specific integer to be between $1$ and $10$ (exclusive), you would write the following constraint:
```python
number = Int('i')
constraint = And(1 < number, number < 10)
```
- ```col_c``` represents the constraints that each value in our array must be [Distinct](https://z3prover.github.io/api/html/z3.z3.html#-Distinct)
- ```diag_c``` represents the constraints that no other queen must be in the diagonal. This is the trickiest constraint to define, as you will require a [conditional expression](https://z3prover.github.io/api/html/z3.z3.html#-If). First, recall that each element in our array is a distinct column. If `i == j`, then `Q[i] == Q[j]`, and we don't need to constrain the values of `Q[i]` or `Q[j]`. Second, if `i != j`, then we need to ensure that `Q[i] - Q[j] != i - j` (going up the diagonal), and `Q[i] - Q[j] != j - i` (going down de diagonal), for all possible indexes of `Q`.

In [5]:
from z3 import Solver, Int, Distinct, If, And, sat
from printBoard import printBoard
s = Solver()

n_queens = 8

# We know each queen must be in a different row.
# So, we represent each queen by a single integer: the column position
Q = [ Int('Q_%i' % (i)) for i in range(n_queens) ]

# Each queen is in a column {1, ... 8 }
val_c = [And(0 <= Q[i], Q[i] < n_queens) for i in range(n_queens)]

# At most one queen per column
col_c = [Distinct(Q)]

# Diagonal constraint
diag_c = [
    If(i ==j,
      True,
    And(Q[i] - Q[j] != i - j, Q[i] - Q[j] != j - i))
    for i in range(n_queens) for j in range (i)]

# Mix all constraints together
s.add(val_c + col_c + diag_c)

# And check whether the solver can satisfy our constraints
if s.check() == sat:
    m = s.model()
    result = [[m.evaluate(Q[i]).as_long() for i in range(n_queens)]]
    printBoard(result,0)
else:
    print("failed to solve")

. . . Q . . . .
. Q . . . . . .
. . . . . . . Q
. . . . . Q . .
Q . . . . . . .
. . Q . . . . .
. . . . Q . . .
. . . . . . Q .




### Comparison with Local Search Problems

Go back to your implementation of the N-Queens problem, your implementation of the local search algorithms, and the source code in [nqueens.py](../week04/nqueens.py). How large is your description of the constraints in this practical versus the entire code of the problem description and the search procedure?

## Sudoku Problems

[Sudoku](https://en.wikipedia.org/wiki/Sudoku) is a popular type of puzzle which is particularly suitable for solving using a CSP formulation. The goal is to insert the numbers in the boxes to satisfy only one condition: each row, column and $3 \times 3$ box must contain the digits $1$ through $9$ exactly once.

![n-Queens](img/sudoku-example.svg "A sequence of valid moves for sliding-puzzle's navigation problem")

The code below should encode the constraints for a sudoku problem in Z3. Different sudoku instances can be solved by modifying the matrix ```instance```.

In [7]:
from z3 import print_matrix
# Sudoku using CSP

# 9x9 matrix of integer variables
X = [ [ Int("x_%s_%s" % (i+1, j+1)) for j in range(9) ]
      for i in range(9) ]

# each cell contains a value in {1, ..., 9}
cells_c  = [And(0 <= [i], [i] < 9) for i in range(9)]

# each row contains a digit at most once
rows_c   = # YOUR CODE HERE

# each column contains a digit at most once
cols_c   =

# each 3x3 square contains a digit at most once
sq_c     = # YOUR CODE HERE

sudoku_c = cells_c + rows_c + cols_c + sq_c

# sudoku instance, we use '0' for empty cells
instance = ((5,3,0,0,7,0,0,0,0),
            (6,0,0,1,9,5,0,0,0),
            (0,9,8,0,0,0,0,6,0),
            (8,0,0,0,6,0,0,0,3),
            (4,0,0,8,0,3,0,0,1),
            (7,0,0,0,2,0,0,0,6),
            (0,6,0,0,0,0,2,8,0),
            (0,0,0,4,1,9,0,0,5),
            (0,0,0,0,8,0,0,7,9))

instance_c = [ If(instance[i][j] == 0,
                  True,
                  X[i][j] == instance[i][j])
               for i in range(9) for j in range(9) ]

s = Solver()
s.add(sudoku_c + instance_c)
if s.check() == sat:
    m = s.model()
    r = [ [ m.evaluate(X[i][j]) for j in range(9) ]
          for i in range(9) ]
    print_matrix(r)
else:
    print("failed to solve")

SyntaxError: invalid syntax (ipython-input-625825797.py, line 12)